In [ ]:
import numpy as np
import pandas as pd
import copy
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score,  roc_auc_score, roc_curve,confusion_matrix,cohen_kappa_score,precision_score, recall_score
from sklearn.model_selection import train_test_split,cross_val_score,RepeatedStratifiedKFold,GridSearchCV
from tqdm import tqdm_notebook as tqdm
from google.cloud.bigquery import Client, QueryJobConfig
import warnings
import random
warnings.filterwarnings("ignore")

In [ ]:
Query_Start_time = datetime.datetime.now()
query1 = """WITH KEEP_STORE_SKU AS (
SELECT store_sku
FROM (
    SELECT *
    FROM (
            SELECT concat(STORE_NUM,'-',SKU) store_sku,
                   OOO_FLAG as flag,
                   count(1) over (partition by concat(STORE_NUM,'-',SKU),OOO_FLAG) ct,
                   count(1) over (partition by concat(STORE_NUM,'-',SKU)) tot_ct
            FROM `table`
    )
    PIVOT (ANY_VALUE(ct) Flag FOR Flag IN ('Inv', 'StockOut'))
)
WHERE (tot_ct!=flag_Inv OR tot_ct!=flag_StockOut) and Flag_Inv >1
),

FINAL_DATA AS (
SELECT DISTINCT POST_DATE,
       SKU,
       STORE_NUM,
       OUTLET_STRATEGY,
       PRODUCT_KEY,
       DAYOFWEEK_NM,
       BRAND,
       DEPT_STORE_GRADE,
       DEPARTMENT_ID,
       DEPARTMENT_DESC,
       DIVISION_ID,
       DIVISION_DESC,
       BRAND_STORE_FORMAT,
       CC,
       CLASS_ID,
       CLASS_DESC,
       COLOR_ID,
       COLOR_DESC,
       STYLE_ID,
       STYLE_DESC,
       SIZE_DESC,
       SIZE_ID,
       STR_COUNTRY,
       STR_STATE,
       FISCAL_WEEK,
       INV_OH_UT_QN,
       SLS_UT_QN,
       OOO_FLAG,
       LIFE_CYCLE_FLAG,
       ITEM_CYCLE_TYPE,
       DEPARTMENT_INVENTORY,
       DEPARTMENT_SALES,
       CLASS_INVENTORY,
       CLASS_SALES,
       STYLE_INVENTORY,
       STYLE_SALES,
       STORE_CLASS_SI,
       STORE_DEPT_SI,
       STORE_STYLE_SI,
       DPT_GRADE_CLASS_SI,
       DPT_GRADE_DEPT_SI,
       DPT_GRADE_STYLE_SI
FROM `Sorce table`
)

SELECT * from final_data
WHERE product_key in (SELECT STORE_SKU FROM KEEP_STORE_SKU)
AND DEPARTMENT_ID IN (1,2,3,4,5)"""

client = Client('schema')
job1 = client.query(query1)
df = job1.to_dataframe()
Query_End_time = datetime.datetime.now()
Query_Run_Time = Query_End_time-Query_Start_time
print(Query_Run_Time)

In [ ]:
ls_data = df
ls_data['Pred'] = ls_data['SLS_UT_QN'].astype(int).astype(str)
dummy = ls_data.pop('DAYOFWEEK_NM')
ls_data = pd.concat([ls_data,pd.get_dummies(dummy)],axis=1)
product_list = list(ls_data['PRODUCT_KEY'].unique())
len(product_list)

In [ ]:
grouped_data = ls_data.groupby('PRODUCT_KEY')

# Create a list of DataFrames using dictionary comprehension
data_groups = [group_df.copy() for _, group_df in grouped_data]

In [ ]:
def model_fit(chk):
    try:
        #print(chk['PRODUCT_KEY'].unique())
        #chk = ls_data[ls_data['PRODUCT_KEY']==product_list[0]]
        chk_test = chk[chk['OOO_FLAG'] == 'StockOut']
        chk_train = chk[chk['OOO_FLAG'] == 'Inv']
        label_train = chk_train.pop("Pred")
        label_test = pd.DataFrame(chk_test.pop("Pred"))
        cols_selected = ['DEPARTMENT_INVENTORY','STYLE_SALES','DPT_GRADE_CLASS_SI','Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday','Wednesday']
        #'days_to_EOL'
        x_train, x_val, y_train, y_val = train_test_split(chk_train[cols_selected], label_train,test_size=0.1,random_state=0)#,stratify = label_train
        # Adding Train/Test/Val Flags to data
        chk.loc[chk_test.index, 'Train_Test_Val'] = 'Test'
        chk.loc[x_train.index, 'Train_Test_Val'] = 'Train'
        chk.loc[x_val.index, 'Train_Test_Val'] = 'Val'
        rf = RandomForestClassifier()
        mdl2 = rf.fit(x_train,y_train)
        pred_y_val = mdl2.predict(x_val[cols_selected])
        pred_y_val = pd.DataFrame(pred_y_val).rename(columns={0:'Predicted'})
        pred_y_val.index = y_val.index
        chk.loc[chk.index.isin(pred_y_val.index),['Pred']] = pred_y_val['Predicted']
        Accuracy = accuracy_score(y_val,pred_y_val)
        Kappa= cohen_kappa_score(y_val,pred_y_val)
        F1_Macro = f1_score(y_val,pred_y_val,average='macro')
        y_pred = rf.predict(chk_test[cols_selected])
        y_pred = pd.DataFrame(y_pred).rename(columns={0:'Predicted'})
        y_pred.index = label_test.index
        chk.loc[chk.index.isin(y_pred.index),['Pred']] = y_pred['Predicted'] #getting the Full dataset with predicted values
        chk['DAYOFWEEK_NM'] = (chk.loc[:, ['Friday','Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']] == 1).idxmax(1)
        chk = chk.drop(columns=[col for col in chk if col.startswith(('DAYOFWEEK_NM_','ITEM_CYCLE_TYPE_'))])
        op_cols = ['POST_DATE', 'SKU', 'CC','STORE_NUM','PRODUCT_KEY', 'DAYOFWEEK_NM','BRAND','DEPT_STORE_GRADE','DEPARTMENT_ID', 'DEPARTMENT_DESC','DIVISION_ID','DIVISION_DESC', 'BRAND_STORE_FORMAT', 'CLASS_ID','CLASS_DESC', 'COLOR_ID', 'COLOR_DESC', 'STYLE_ID', 'STYLE_DESC','SIZE_ID', 'SIZE_DESC', 'STR_COUNTRY', 'STR_STATE', 'FISCAL_WEEK', 'OOO_FLAG', 'LIFE_CYCLE_FLAG', 'Train_Test_Val','ITEM_CYCLE_TYPE','INV_OH_UT_QN', 'SLS_UT_QN', 'Pred']
        chk = chk[op_cols]
        y_pred_prob = rf.predict_proba(chk_test[cols_selected])
        y_pred_prob = pd.DataFrame(y_pred_prob)
        y_pred_prob.columns = ['Prob_' + str(col) for col in y_pred_prob.columns]
        y_pred_prob.index = label_test.index
        Predictions_df = pd.concat([chk.loc[y_pred_prob.index,['POST_DATE', 'SKU', 'STORE_NUM']],y_pred,y_pred_prob],axis=1)
        ans = chk.merge(Predictions_df, how='left', on=['POST_DATE', 'SKU', 'STORE_NUM']).drop(columns=['Predicted'])
        ans[['Accuracy', 'Kappa','F1_Macro']] = Accuracy,Kappa,F1_Macro
        ans['Kappa']=ans['Kappa'].fillna(1)
        return ans
    except:
        print(chk['PRODUCT_KEY'].unique())

In [ ]:
#data_groups_samp = data_groups[(len(data_groups)//2):]

In [ ]:
import concurrent.futures
import multiprocessing
import psutil
import time
import datetime
import pandas as pd
from tqdm import tqdm


def process_data(dgs, executor):
    model_output = pd.DataFrame()  # Initialize an empty dataframe
    with tqdm(total=len(dgs), desc="Batch_Progress") as pbar:
        for result_df in executor.map(model_fit, dgs):
            model_output = pd.concat([model_output, result_df], ignore_index=True)
            pbar.update(1)
    return model_output

if __name__ == "__main__":
    start_time = datetime.datetime.now()
    master_output = pd.DataFrame()  # Initialize the master output dataframe

    batch_size = 1000
    total_batches = (len(data_groups) + batch_size - 1) // batch_size  # Calculate the total number of batches
    print("total_batches Count:", total_batches)

    # Create the ProcessPoolExecutor with the desired number of processes
    with concurrent.futures.ProcessPoolExecutor(16) as executor:
        for i in range(total_batches):
            batch_data_groups = data_groups[i * batch_size: (i + 1) * batch_size]  # Extract a batch of data_groups
            print("Batch", i)

            # Process the batch and get the results
            batch_output = process_data(batch_data_groups, executor)

            # Append the batch results to the master output dataframe
            master_output = pd.concat([master_output, batch_output], ignore_index=True)

    end_time = datetime.datetime.now()
    run_time = end_time - start_time
    print("Total run time:", run_time)

In [ ]:
master_output['Max_Prob'] = master_output[[col for col in master_output if col.startswith('Prob_')]].max(axis=1)
master_output['SLS_UT_QN']=master_output['SLS_UT_QN'].astype(int)
master_output['Pred']=master_output['Pred'].astype(int)
#master_output['Model'] = 'SKU_STR_DCSINV_DCSSales_DCSSI_DCSGradeSI_no_max_features'
master_output['Model'] = 'SKU_STR_DINV_SSales_DGradeSI_no_max_features'

In [ ]:
#master_output.columns,master_output['Model'].unique()
master_output.columns

Index(['POST_DATE', 'SKU', 'CC', 'STORE_NUM', 'PRODUCT_KEY', 'DAYOFWEEK_NM',
       'BRAND', 'DEPT_STORE_GRADE', 'DEPARTMENT_ID', 'DEPARTMENT_DESC',
       'DIVISION_ID', 'DIVISION_DESC', 'BRAND_STORE_FORMAT', 'CLASS_ID',
       'CLASS_DESC', 'COLOR_ID', 'COLOR_DESC', 'STYLE_ID', 'STYLE_DESC',
       'SIZE_ID', 'SIZE_DESC', 'STR_COUNTRY', 'STR_STATE', 'FISCAL_WEEK',
       'OOO_FLAG', 'LIFE_CYCLE_FLAG', 'Train_Test_Val', 'ITEM_CYCLE_TYPE',
       'INV_OH_UT_QN', 'SLS_UT_QN', 'Pred', 'Prob_0', 'Prob_1', 'Accuracy',
       'Kappa', 'F1_Macro', 'Prob_2', 'Prob_3', 'Prob_4', 'Prob_5', 'Prob_6',
       'Prob_7', 'Prob_8', 'Max_Prob', 'Model'],
      dtype='object')

In [1]:
from google.cloud import bigquery
client = bigquery.Client(project = 'schema')
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

#df is the dataframe to upload, {project_id}.{dataset_id}.{table_name} is the table id in which you would like upload that dataframe
client.load_table_from_dataframe(master_output, 'schema.table', job_config=job_config)